In [ ]:
import os
import numpy as np
import re
from statannotations.Annotator import Annotator
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

In [ ]:
assert int(pd.__version__[0]) < 2, 'Please < 2 required for statannotations'

In [ ]:
data_dir = '/Users/jk1/temp/cereblink/data_saving/exclude_nan_outcome_False'
pvals_dir = '/Users/jk1/temp/cereblink/data_saving/exclude_nan_outcome_False'
output_dir = '/Users/jk1/Downloads'

In [ ]:
data_filenames = [f for f in os.listdir(data_dir) if f.endswith('.csv') and 'timebin' in f and 'reassembled_pupillometry' in f]

pupillometry_df = pd.DataFrame()
for data_filename in data_filenames:
    # find timebin size with regex identifying pattern : _xh_
    timebin_size = int(re.search(r'_(\d+)h_', data_filename).group(1))
    data_is_normalized = int(('normalized' in data_filename) or ('normalised' in data_filename))
    outcome = '_'.join(data_filename.split('_')[0:2])

    df = pd.read_csv(os.path.join(data_dir, data_filename))
    df['timebin_size'] = timebin_size
    df['normalized'] = data_is_normalized
    df['outcome'] = outcome
    pupillometry_df = pd.concat([pupillometry_df, df], axis=0)
    
pupillometry_df = pupillometry_df.reset_index(drop=True)

In [ ]:

pvals_filenames = [f for f in os.listdir(pvals_dir) if f.endswith('.csv') and 'pvals' in f]

pvals_df = pd.DataFrame()
for pvals_filename in pvals_filenames:
    # find timebin size with regex identifying pattern : _xh_
    timebin_size = int(re.search(r'_(\d+)h_', pvals_filename).group(1))
    data_is_normalized = int(('normalized' in pvals_filename) or ('normalised' in pvals_filename))
    using_span = int(('with_span' in pvals_filename))
    outcome = '_'.join(pvals_filename.split('_')[0:2])

    df = pd.read_csv(os.path.join(pvals_dir, pvals_filename), index_col=0)
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'metric'}, inplace=True)
    df['timebin_size'] = timebin_size
    df['normalized'] = data_is_normalized
    df['using_span'] = using_span
    df['outcome'] = outcome
    pvals_df = pd.concat([pvals_df, df], axis=0)

pvals_df = pvals_df[pvals_df['using_span'] == 0]

In [ ]:
selected_colors_palette = sns.color_palette(['#E6F5F5', '#F8F0FF'], n_colors=2)
selected_colors_palette

In [ ]:
pupillometry_metrics = ['NPI', 'CV']
inter_eye_metrics = ['mean', 'min', 'max', 'delta']
# combine to get all metrics
single_timepoint_metrics = [f'{metric}_inter_eye_{metric_type}' for metric in pupillometry_metrics for metric_type in
                            inter_eye_metrics]
over_time_metrics = ['max', 'min', 'median']
# combine to get all metrics
timebin_metrics = [f'{metric}_timebin_{metric_type}' for metric in single_timepoint_metrics for metric_type in
                   over_time_metrics]

In [ ]:
pvals_df.head()

In [ ]:
target_df = pupillometry_df[(pupillometry_df['outcome'] == 'DCI_ischemia') & (pupillometry_df['normalized'] == 1) & (pupillometry_df['timebin_size'] == 8)]
metric = 'CV_inter_eye_min_timebin_max'

In [ ]:
pvals_metric = pvals_df[(pvals_df['metric'] == metric) & (pvals_df['outcome'] == 'DCI_ischemia') & (pvals_df['normalized'] == 1) & (pvals_df['timebin_size'] == 8)]
pvals_metric

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 7.5))
sns.set_theme(style="whitegrid", context="paper", font_scale = 8)
# box plot lines in black

plot_params = {
                'data': target_df,
                'x': 'timebin_size',
                'y': metric,
                'hue': 'label',
                'palette': selected_colors_palette,
                'showfliers': False,
                'boxprops':{'edgecolor':'black'},
                'medianprops':{'color':'black'},
                'whiskerprops':{'color':'black'},
                'capprops':{'color':'black'}
            }

sns.boxplot(**plot_params, ax=ax)

pairs = (
                [(8, 0), (8, 1)],
            )

annotator = Annotator(ax,pairs , **plot_params, verbose=False)
annotator.set_pvalues(pvals_metric.adjusted_pval.values)
annotator.annotate()

ax.set_xlabel('')
ax.set_xticklabels([''])
ax.set_ylabel('norm(CV)', fontstyle='italic')
# set legend title
# ax.legend(title='DCI', loc='upper left')
# remove legend
ax.get_legend().remove()

In [ ]:
# fig.savefig(os.path.join(output_dir, f'{metric}_DCI_ischemia_normalized_8h.svg'), format='svg', bbox_inches='tight')